In [33]:
import pandas as pd
import numpy as np
import re 
import matplotlib.pyplot as plt



df = pd.read_csv("2000-01_pbp.csv")
print (df.columns)
df['PLAYER1_ID'].value_counts().to_csv("player1_id_counts.csv")
df[df['PLAYER1_ID'] == 406].to_csv("player_406_2000-01_pbp.csv", index=False)



Index(['Unnamed: 0', 'EVENTMSGACTIONTYPE', 'EVENTMSGTYPE', 'EVENTNUM',
       'GAME_ID', 'HOMEDESCRIPTION', 'NEUTRALDESCRIPTION', 'PCTIMESTRING',
       'PERIOD', 'PERSON1TYPE', 'PERSON2TYPE', 'PERSON3TYPE', 'PLAYER1_ID',
       'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'PLAYER1_TEAM_CITY',
       'PLAYER1_TEAM_ID', 'PLAYER1_TEAM_NICKNAME', 'PLAYER2_ID',
       'PLAYER2_NAME', 'PLAYER2_TEAM_ABBREVIATION', 'PLAYER2_TEAM_CITY',
       'PLAYER2_TEAM_ID', 'PLAYER2_TEAM_NICKNAME', 'PLAYER3_ID',
       'PLAYER3_NAME', 'PLAYER3_TEAM_ABBREVIATION', 'PLAYER3_TEAM_CITY',
       'PLAYER3_TEAM_ID', 'PLAYER3_TEAM_NICKNAME', 'SCORE', 'SCOREMARGIN',
       'VISITORDESCRIPTION', 'WCTIMESTRING'],
      dtype='object')


Shaq 00-01 Season Discovery

In [ ]:
shaq = df[df['PLAYER1_ID'] == 406]
shaq_scoring = shaq[shaq['EVENTMSGTYPE'].isin([1,3])]
descriptions = shaq_scoring['HOMEDESCRIPTION'].fillna('') + shaq_scoring['VISITORDESCRIPTION'].fillna('')  
def get_points(text):
    if '3PT' in text:
        return 3
    elif '2PT' in text:
        return 2
    elif 'FREE THROW' in text:
        return 1
    else:
        return 0

points = descriptions.apply(get_points)
shaq_scoring['points'] = points
points_by_game = shaq_scoring.groupby('GAME_ID')['points'].sum()
best_game_id = points_by_game.idxmax()
best_game_id = points_by_game.idxmax()
best_game_points = points_by_game.max()

print(f"Shaq's best game in 2000-01 was game {best_game_id} with {best_game_points} points.")   



In [35]:
import re

def extract_points(text):
    if not isinstance(text, str):
        return 0
    match = re.search(r"\((\d+) PTS\)", text)
    if match:
        return int(match.group(1))
    return 0

# 1. Filter to Shaq and copy
shaq = df[df['PLAYER1_ID'] == 406].copy()

# 2. Build description text
desc = shaq['HOMEDESCRIPTION'].fillna('') + " " + shaq['VISITORDESCRIPTION'].fillna('')

# 3. Extract points per scoring play (running total)
shaq['points'] = desc.apply(extract_points)

# 4. Get final total per game (max, not sum)
points_by_game = shaq.groupby('GAME_ID')['points'].max().sort_values(ascending=False)

print(points_by_game.head(10))



GAME_ID
20000571    41
20000998    40
20001088    39
20001083    39
20000049    39
20000739    38
20000807    37
20000230    36
20000981    36
20000012    36
Name: points, dtype: int64


In [36]:
# 1. Filter to Shaq and copy
shaq = df[df['PLAYER1_ID'] == 406].copy()

# 2. Build description text
desc = shaq['HOMEDESCRIPTION'].fillna('') + " " + shaq['VISITORDESCRIPTION'].fillna('')

# 3. Extract points per scoring play (running total)
shaq['points'] = desc.apply(extract_points)

# 4. Get final total per game (max, not sum)
points_by_game = shaq.groupby('GAME_ID')['points'].max().sort_values(ascending=False)

# 5. Set Shaq - assists are PLAYER2_ID on made baskets (EVENTMSGTYPE = 1)
shaq = df[(df['PLAYER2_ID'] == 406) & (df['EVENTMSGTYPE'] == 1)].copy()

# 6. Build description text
desc = shaq['HOMEDESCRIPTION'].fillna('') + " " + shaq['VISITORDESCRIPTION'].fillna('')

# 7. Extract assists per play - each row is 1 assist by Shaq
shaq['assists'] = 1

# 8. Get final total per game (sum)
assists_by_game = shaq.groupby('GAME_ID')['assists'].sum().sort_values(ascending=False)

#9. Set Shaq
shaq = df[df['PLAYER1_ID'] == 406].copy()

#10. Build description text
desc = shaq['HOMEDESCRIPTION'].fillna('') + " " + shaq['VISITORDESCRIPTION'].fillna('')

#11. Extract rebounds per play
shaq = df[df['PLAYER1_ID'] == 406].copy()
desc = shaq['HOMEDESCRIPTION'].fillna('') + " " + shaq['VISITORDESCRIPTION'].fillna('')
shaq['rebounds'] = desc.str.contains("REBOUND").astype(int)
rebounds_by_game = shaq.groupby('GAME_ID')['rebounds'].sum().sort_values(ascending=False)

# 12. Get final total per game (sum)
rebounds_by_game = shaq.groupby('GAME_ID')['rebounds'].sum().sort_values(ascending=False)

#13 Set Shaq 
shaq = df[df['PLAYER3_ID'] == 406].copy()

#14 Build description text
desc = shaq['HOMEDESCRIPTION'].fillna('') + " " + shaq['VISITORDESCRIPTION'].fillna('')

#15 Extract blocks per play
shaq = df[df['PLAYER3_ID'] == 406].copy()
shaq['blocks'] = 1
blocks_by_game = shaq.groupby('GAME_ID')['blocks'].sum()


# 16. Get final total per game (sum)
blocks_by_game = shaq.groupby('GAME_ID')['blocks'].sum().sort_values(ascending=False)

# 17 Set Shaq - steals are recorded with PLAYER2_ID (the person who stole the ball)
# and EVENTMSGTYPE = 5 (turnover/steal events)
shaq = df[(df['PLAYER2_ID'] == 406) & (df['EVENTMSGTYPE'] == 5)].copy()

# 18 Build description text
desc = shaq['HOMEDESCRIPTION'].fillna('') + " " + shaq['VISITORDESCRIPTION'].fillna('')

# 19 Extract steals per play - each row is 1 steal by Shaq
shaq['steals'] = 1

# 20. Get final total per game (sum)
steals_by_game = shaq.groupby('GAME_ID')['steals'].sum().sort_values(ascending=False)

# Combine all stats into a single DataFrame
shaq_stats = pd.DataFrame({
    'points': points_by_game,
    'assists': assists_by_game,
    'rebounds': rebounds_by_game,
    'blocks': blocks_by_game,
    'steals': steals_by_game
}).fillna(0)
print(shaq_stats.head(10))

          points  assists  rebounds  blocks  steals
GAME_ID                                            
20000012      36      5.0        11     4.0     1.0
20000019      34      1.0        15     0.0     2.0
20000047      27      5.0        15     4.0     1.0
20000049      39      4.0        14     5.0     0.0
20000058      24      5.0         7     3.0     0.0
20000068      13      4.0        17     1.0     0.0
20000098      14      3.0         5     6.0     0.0
20000108      34      2.0        19     2.0     1.0
20000124      33      4.0        16     4.0     1.0
20000147      14      5.0         9     1.0     0.0


In [37]:
# Compute Game Score-like metric using available columns in shaq_stats
# Missing shooting stats default to 0 if not present
shaq_stats['gamescore'] = (
	shaq_stats['points']
	+ 0.4 * shaq_stats.get('field_goals_made', pd.Series(0, index=shaq_stats.index))
	- 0.7 * shaq_stats.get('field_goals_attempted', pd.Series(0, index=shaq_stats.index))
	+ 0.7 * shaq_stats.get('free_throws_made', pd.Series(0, index=shaq_stats.index))
	- 0.4 * shaq_stats.get('free_throws_attempted', pd.Series(0, index=shaq_stats.index))
	+ 0.6 * shaq_stats['rebounds']
	+ shaq_stats['steals']
	+ 0.7 * shaq_stats['assists']
	+ 0.7 * shaq_stats['blocks']
)

print(shaq_stats.sort_values(by='gamescore', ascending=False).head(10))

          points  assists  rebounds  blocks  steals  gamescore
GAME_ID                                                       
20000998      40      8.0        17     5.0     3.0       62.3
20000807      37      6.0        19     5.0     3.0       59.1
20000049      39      4.0        14     5.0     0.0       53.7
20000519      34      4.0        23     2.0     1.0       53.0
20000739      38      4.0        12     5.0     1.0       52.5
20000753      35      7.0        15     2.0     1.0       51.3
20001088      39      1.0        14     2.0     1.0       50.5
20001083      39      5.0        10     2.0     0.0       49.9
20000012      36      5.0        11     4.0     1.0       49.9
20000571      41      2.0        11     1.0     0.0       49.7


Uses gamescore equation to find Shaq's best games in the 00-01 Season.